## Teoría de perturbaciones

Consiste en resolver un sistema perturbado(se conoce la solución al no perturbado), y donde el interés es conocer la contribución de la parte perturbada $H'$ al nuevo sistema total. 

$$ H = H^{0} + H'$$

Para sistemas no degenerados, la corrección a la energía a primer orden se calcula como 

$$ E_{n}^{(1)} = \int\psi_{n}^{(0)*} H' \psi_{n}^{(0)}d\tau$$

** Tarea 1 : Programar esta ecuación si conoces $H^{0}$ y sus soluciones. **

In [1]:
### Solución
from sympy import *
from sympy import init_printing; init_printing(use_latex = 'mathjax')
import numpy as np
from sympy.physics.quantum.constants import hbar
import scipy.integrate as integ
import scipy.constants as cons
var('x r phi theta r_1 r_2 phi_1 phi_2 theta_1 theta_2 gamma')
var('n l m', positive = 'True', integer = 'True')
var('m_e Z a_0 q L epsilon_0 omega', constant = 'True')
var('c d', real = 'True', constant = 'True')
import sympy.physics.hydrogen as hyd
import sympy.functions.special.spherical_harmonics as sph
import sympy.physics.qho_1d as qho

In [2]:
#Átomo de Hidrogeno
def Hidrogeno():
    Yhyd = hyd.R_nl(n, l, r, Z)
    Arm = sph.Ynm(m, l, theta, phi)  #Función de onda átomo de hidrógeno
    Ehyd = hyd.E_nl(n, Z) * (hbar ** 2 / (a_0 ** 2 * m_e)) #Energía en eV
    return Yhyd, Arm, Ehyd

In [3]:
#Oscilador armónico 1D
#H = c * x ** 3 + d * x ** 4
def oscilador():
    Eqho = qho.E_n(n, omega)
    Yqho = qho.psi_n(n, x, m, omega)
    return Yqho, Eqho

In [4]:
#Particula en una caja 1D (box)
#H = x ** 2
def caja():
    Ebox = (hbar * pi * n) ** 2 / ((2* m_e) * L **2)
    Ybox = sqrt(2 / L) * sin(n * pi * x / L)
    return Ybox, Ebox

In [5]:
#General y Menú
print('Para que sistema desea resolver: 1. Oscilador armónico 1D, 2. Partícula en una caja 1D, 3. Atómo de Hidrogeno')
sis = input('Introduzca el número de opción')
print('Ahora toca elegir la perturbación H´ (para el atomo de hidrogeno (corrección relativista) presione cualquier tecla)')
H = sympify(input('Introduzca la perturbación'))
aprox = 5 #que aproximación quieres + 1
if sis == '1':
    Y0, E0 = oscilador()
    nbase = 0
    linf = -oo
    lsup = oo
    ww = 1e15
    def psis():
        EqhoC = integrate((Y0* H * Y0).subs(n, nbase), (x, linf, lsup)).args[0][0]
        YqhoS = np.asarray([(integrate((Y0.subs(n, i)* H * Y0.subs(n, nbase)), (x, -oo, oo))).args[0][0] \
                            for i in range(0,aprox) if i != nbase])
        EqhoS = np.asarray([E0.subs([(n, nbase), (omega, ww), (hbar, cons.hbar)]) \
                            - E0.subs([(n, i), (omega, ww), (hbar, cons.hbar)]) for i in range(0,aprox) if i != nbase])
        Cqho = YqhoS / EqhoS
        YqhoC = [Cqho[i-nbase-1] * Y0.subs(n, i) for i in range(0, aprox) if i != nbase]
        PsiQho = np.sum(YqhoC)
        Cqho2 = YqhoS ** 2 / EqhoS #Oscilador
        Eqho2 = np.sum(Cqho2)
        return EqhoC, Eqho2, PsiQho
elif sis == '2':
    Y0, E0 = caja()
    nbase = 1
    linf = 0
    lsup = L
    LL = 1e-9
    def inte(x, LL, mm, nn, pii):
        return Y0.subs([(L, LL), (n, nn), (pi, pii)])*H*Y0.subs([(L, LL), (n, mm), (pi, pii)])
    def psis():
        EnBox = (integrate(Y0*H*Y0, (x, linf, lsup))).simplify()
        YboxS = np.asarray([integ.quad(lambdify(x, inte(x, LL, i, nbase, np.pi), 'numpy'), 0, LL) \
                            for i in range(1,aprox) if i != nbase])
        EboxS = np.asarray([E0.subs([(L, LL), (pi, np.pi), (n, nbase), (m_e, cons.m_e), (hbar, cons.hbar)]) \
                            - E0.subs([(L, LL), (pi, np.pi), (n, i), (m_e, cons.m_e), (hbar, cons.hbar)]) \
                            for i in range(1,aprox) if i != nbase])
        Cbox = YboxS[:,0] / EboxS
        YboxC = [Cbox[i-2] * Y0.subs(n, i) for i in range(1, aprox) if i != nbase]
        PsiBox = np.sum(YboxC)
        Cbox2 = (YboxS[:,0]) ** 2 / EboxS #CAJA
        Ebox2 = np.sum(Cbox2)
        return EnBox, Ebox2, PsiBox
elif sis == '3':
    Y0, A0, E0 = Hidrogeno()
    nbase = 1
    lbase = 0
    mbase = 0
    linf = 0
    lsup = oo
    def psis():
        Rad = Y0.subs([(l, lbase), (Z, 1)])
        RadHRad = -Rad.subs(n, nbase) *( 1/ (8*m_e**3*c**2)) * hbar ** 4  * diff(Rad.subs(n, nbase), r, 4)
        EhydC = integrate(RadHRad*r**2, (r, 0, oo))
        EhydC = EhydC.subs([(hbar, cons.hbar), (m_e, cons.m_e), (c, cons.c)]) / (4 * cons.epsilon_0) ** 4
        YhydS = np.asarray([(integrate(-Rad.subs(n, i) *( 1/ (8*m_e**3*c**2)) * hbar ** 4  * diff(Rad.subs(n, nbase), r, 4), (r, 0, oo))) \
                   for i in range(1,aprox) if i != nbase])
        EhydS = np.asarray([E0.subs([(n, nbase), (Z, 1)]) - E0.subs([(n, i), (Z, 1)]) for i in range(1,aprox) if i != nbase])
        Chyd = YhydS / EhydS
        YhydC = [Chyd[i-nbase-1] * Rad.subs([(n, i)]) for i in range(1, aprox) if i != nbase]
        PsiHyd = np.sum(YhydC)
        Chyd2 = YhydS ** 2 / EhydS
        Ehyd2 = np.sum(Chyd2)
        return EhydC, Ehyd2, PsiHyd
else:
    print('Selección inválida')


Para que sistema desea resolver: 1. Oscilador armónico 1D, 2. Partícula en una caja 1D, 3. Atómo de Hidrogeno
Introduzca el número de opción3
Ahora toca elegir la perturbación H´ (para el atomo de hidrogeno (corrección relativista) presione cualquier tecla)
Introduzca la perturbación1


In [6]:
E1, E2, Psi = psis()

In [7]:
Psi

                                                    -r                        
                   -r             ⎛   2          ⎞  ───             ⎛   3    2
                   ───      2   2 ⎜2⋅r           ⎟   3        2   2 ⎜  r    r 
   2   2            2    7⋅ℏ ⋅a₀ ⋅⎜──── - 2⋅r + 3⎟⋅ℯ      34⋅ℏ ⋅a₀ ⋅⎜- ── + ──
2⋅ℏ ⋅a₀ ⋅(-r + 2)⋅ℯ               ⎝ 9            ⎠                  ⎝  48   2 
────────────────────── + ────────────────────────────── + ────────────────────
          2   2                         2   2                               2 
      27⋅c ⋅mₑ                     576⋅c ⋅mₑ                          9375⋅c ⋅

             -r 
          ⎞  ───
          ⎟   4 
 - 3⋅r + 4⎟⋅ℯ   
          ⎠     
────────────────
  2             
mₑ              

In [8]:
E1

-1.44634324025469e-22

In [9]:
E2

              6   2 
-42820217573⋅ℏ ⋅a₀  
────────────────────
               4   5
1944000000000⋅c ⋅mₑ 

Y la corrección a la función de onda, también a primer orden, se obtiene como:

 $$ \psi_{n}^{(1)} = \sum_{m\neq n} \frac{\langle\psi_{m}^{(0)} | H' | \psi_{n}^{(0)} \rangle}{E_{n}^{(0)} - E_{m}^{(0)}} \psi_{m}^{(0)}$$

**Tarea 2: Programar esta ecuación si conoces $H^{0}$ y sus soluciones. **

In [10]:
### Solución en tarea 1

**Tarea 3: Investigue las soluciones a segundo orden y también programe las soluciones. **

In [11]:
### Solución en tarea 1

**Tarea 4.  Resolver el átomo de helio aplicando los programas anteriores.** 

In [38]:
var('r_12')
#Obtencion perturbación átomo He
H = 1 / r_12
#r12 = sqrt(r_1**2 + r_2**2 - 2 * r_1 * r_2 * cos(gamma))
#parte Radial
Rad = hyd.R_nl(1, 0, r, 2)
#Numérico (Revisar)
#Irad = (Rad1 * Rad2 * H * Rad1 * Rad2).subs([(q, cons.e), (epsilon_0, cons.epsilon_0), (pi, np.pi), (r_1-r_2, r12)])
#Ir = lambdify((r_1, r_2, gamma), Irad)
#IRAD = integ.nquad(Ir, [[0, np.inf], [0, np.inf], [0, np.pi]])
#EHe = IRAD[0] * ( 1/2) #Multiplicar por 1 / (cons.epsilon_0) para las unidades
I1 = integrate((Rad*H*Rad* r ** 2).subs([(r, r_1), (r_12, r_2)]), (r_1, 0, r_2)) + \
                integrate((Rad*H*Rad* r ** 2).subs([(r, r_1), (r_12, r_1)]), (r_1, r_2, oo))
EHe = integrate((Rad*Rad * r ** 2).subs(r, r_2) * (I1), (r_2, 0, oo)) / 2  #para pasar de Hartree a eV

In [88]:
EHe * (2 * 27.2114) #para pasar de Hartree a eV

34.0142500000000

**Tarea 5: Método variacional-perturbativo. **

Este método nos permite estimar de forma precisa $E^{(2)}$ y correcciones perturbativas de la energía de órdenes más elevados para el estado fundamental del sistema, sin evaluar sumas infinitas. Ver ecuación 9.38 del libro. 

In [75]:
var('zeta A')
ph = hyd.R_nl(1, 0, r_1, zeta) * hyd.R_nl(1, 0, r_2, zeta)
H =  (zeta - Z)*(1/ r_1 + 1 / r_2)

In [86]:
In1 = integrate(ph * H * ph * r_1**2 * r_2 ** 2, (r_1, 0, oo)).args[0][0]
Sand1 = integrate(In1, (r_2, 0, oo)).args[0][0] + EHe * zeta - zeta ** 2
ZetaVal = solve((diff(Sand1, zeta, 1)), zeta)[0]
Correc = Sand1.subs(zeta, ZetaVal).simplify()
Correc.subs(Z, 2) * 27.2114 #para poner las unidades de eV

-77.4887132812500

**Resolver el átomo de helio, considerando este método (sección 9.4), como mejor le parezca. **

**Tarea 6. Revisar sección 9.7. **

Inicialmente a mano, y sengunda instancia favor de intentar programar sección del problema, i.e. integral de Coulomb  e integral de intercambio.

## Siguiente: Segunda parte, Octubre
Simetrías moleculares y Hartree-Fock